- star detection
    - convert to grayscale
        + and blur (for better robustness?)
        - convert to float32, and divide by maximum value
    + [ ] remove hot pixel (median filter)
    - wavelet transform
        - noise reduction (remove small-scale wavelet layer)
        - remove background (remove large-scale wavelet layer)
        - inverse wavelet transform
    - detect
        - threshold image to binary
            - [ ] auto tune threshold paremeter
        - find contours
            - outer boundary: relative brightness with local background (want relatively large boundary to include all pixels for intensity centroid calculation)
            - filter by brightness
                - too dim: peak brightness
                - [ ] capped brightness (not so useful in star matching)
            - [ ] filter non-circular object: shape (`cv.minAreaRect()`)
            - [ ] filter by size
        - characterization
            - find contour centroid
                - calculate actual intensity centroid
            - denote brightness
- star match
    - star field structure
- star align
    - calculate transform matrix

In [1]:
from __future__ import annotations

In [2]:
import os

In [3]:
# set tiff input folder
dir_input_tiff = os.path.normpath(os.path.join(
    os.getcwd(),
    '../', 'input_tiff/size_full',
))

# sort input files
tiff_list = os.listdir(dir_input_tiff)
tiff_list.sort()
# take the sequence middle as the reference frame
reference_tiff = tiff_list[len(tiff_list) // 2]
# and remove it from the 'to align' list
tiff_list.pop(len(tiff_list) // 2)

'__T_0523.TIF'

In [4]:
import numpy as np
import cv2 as cv

In [5]:
# read tiff file into numpy array using opencv
refe = cv.imread(
    os.path.join(dir_input_tiff, reference_tiff),
    cv.IMREAD_UNCHANGED,
)

## Star Detection

### convert to grayscale

In [6]:
refe_gray = cv.cvtColor(refe, cv.COLOR_BGR2GRAY)  # grayscale
refe_blur = cv.GaussianBlur(refe_gray, (9, 9), 0, 0)  # blur

refe_b_float = refe_blur.astype(np.float32) / np.iinfo(refe_blur.dtype).max
refe_float = refe_gray.astype(np.float32) / np.iinfo(refe_gray.dtype).max

### wavelet transform

In [7]:
import pywt

In [8]:
def wavelet_dec_red_rec(
    image: np.ndarray,
    level: int = 5,
    remove_to_small_scale_layer: int = -1,
    remove_large_scale: bool = True,
) -> np.ndarray:
    # decomposition
    coeffs = pywt.wavedec2(image, 'db8', level=level)
    # reduction
    for n in tuple(range(remove_to_small_scale_layer, 0)):
        for i in tuple(range(0, 3)):
            coeffs[n][i].fill(0)
    if remove_large_scale:
        coeffs[0].fill(0)
    # reconstruction
    return pywt.waverec2(coeffs, 'db8')

In [9]:
refe_b_wlred = wavelet_dec_red_rec(refe_b_float)
refe_wlred = wavelet_dec_red_rec(refe_float)

### detect

In [10]:
# threshold
refe_b_binary = cv.threshold(
    (t := refe_b_wlred),
    t.min() + (t.max() - t.min()) * 0.4,
    255,
    cv.THRESH_BINARY,
)[1].astype(np.uint8)

# find contours
refe_contours = cv.findContours(
    refe_b_binary, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE
)[0]

#### characterization

In [11]:
refe_stars: list[
    tuple[np.ndarray, float]
] = []  # [ (S2A[x, y], intensity) ]

for c in refe_contours:
    # calculate the centroid from clear image, to increase precision
    star_mask = cv.drawContours(
        np.zeros(refe_wlred.shape, refe_wlred.dtype),
        [c], 0, 1, cv.FILLED,
    )
    M = cv.moments(refe_wlred * star_mask)
    centroid = np.array([ M['m10'] / M['m00'], M['m01'] / M['m00'] ])
    # denote brightness from blur image, to increase robustness
    brightness = refe_b_wlred[int(centroid[1]), int(centroid[0])]
    
    refe_stars.append((centroid, brightness))

#### filter and sort by brightness

In [12]:
refe_stars.sort(key=lambda e: e[1], reverse=True)
mean = np.array(tuple(e[1] for e in refe_stars)).mean()
for i in range(len(refe_stars)):
    if refe_stars[i][1] < mean:
        refe_stars = refe_stars[:i]
        break

## Star Match

### star field structure

- find brightest in refe
- calculate distance between brightest and all others
- cut a range in these distances
-
- for each star, find all stars within range
- calculate vector from source to target
- every two vector within angle PI form a triangle, fully characterized by the inner angle and the ratio of the two vector magnitudes (mind the order)
    - take brightest star as angle reference vector
    - for each vector, calculate angle from angle ref. vector (anticlockwise)
    - sort vectors by angle
    - (2 layer) iterate the list, take the anticlockwise angle (later minus former), ratio be the former divided by the later
- so the structure feature array `Sn` would be of the form: `[ Vector1Triangle0, V1T1, V1T2, V2T0, V2T1, V3T0, ... ]`, with each `VnTn` of the form: `[angle, ratio]`
-
- pick 2 structures if categorized as similar, treat the source as the same star
    - pick `S1[i1]` as start, iterate `S2` untile find a match `S2[i2]`
    - pick `S1[i1+1]`, start at `S2[i2+1]`, find match at `S2[i2+n1]`
    - do the same for `S1[i1+2]`, start at `S2[i2+n1+1]`, find match at `S2[i2+n1+n2]`

In [13]:
low, high = np.array([0.1, 1]) * np.array(
    tuple(np.linalg.norm(s[0] - refe_stars[0][0]) for s in refe_stars[1:])
).std()

In [14]:
from typing import TypeVar
T = TypeVar('T')
def iof(l: collections.abc.Sequence[T], i: int) -> T:
    # index overflow / circular linked list
    while True:
        if -1 < i < len(l):
            break
        else:
            i -= (i // len(l)) * len(l)
    return l[i]

In [15]:
refe_structure: list[
    tuple[
        np.ndarray,  # S2A[x, y]; source star centroid
        np.ndarray,  # SNx2A[ [angle, ratio] ]; feature array
    ]
] = []

for s1 in refe_stars:
    neighbour: list[
        list[
            np.ndarray,  # [ S2A(separation vector) ]
            float,  # magnitude of separation vector
            float,  # angle from angle reference vector
        ]
    ] = []

    for s2 in refe_stars:
        if low < np.linalg.norm( (sv := (s2[0] - s1[0])) ) < high:
            # populate separation vector and its magnitude
            neighbour.append([sv, (sr := np.linalg.norm(sv))])
            # Neighbour is already sorted by brightness because `refe_stars`
            # is sorted.  So the `neighbour[0][0]` is the angle reference vector.
            # populate angle from angle reference vector
            neighbour[-1].append(
                np.arccos(
                    np.clip(
                        np.dot(neighbour[0][0], sv) / (neighbour[0][1] * sr),
                        -1, 1
                    )
                ) * np.sign( np.cross(neighbour[0][0], sv) )
            )
    # too few neighbour do not form enough triangles
    if len(neighbour) < 3:
        continue

    neighbour.sort(key=lambda e: e[2])
    feature: list[tuple[float, float]] = []  # [ (angle, ratio) ]
    for i1 in range(0, len(neighbour)):
        for i2 in range(i1+1, i1+len(neighbour)):
            # here we may cross the 'PI, -PI' boundary, then `later - former`
            # becomes the clockwise angle (negative value), so we need to
            # prepare `later` to be greater
            later, former = iof(neighbour, i2)[2], neighbour[i1][2]
            if later < former:
                later += 2 * np.pi
            angle = later - former
            # any angle less than PI is ok, but we take 4/5 PI
            if ((4/5) * np.pi) < angle:
                break
            ratio = neighbour[i1][1] / iof(neighbour, i2)[1]
            feature.append( (angle, ratio) )

    refe_structure.append( (s1[0], np.array(feature)) )

In [16]:
len(refe_stars), len(refe_structure)

(124, 124)

In [17]:
refe_structure[3]

(array([3862.61713559,  735.7704727 ]),
 array([[2.25925358e-01, 4.44248036e-01],
        [2.26805324e-01, 4.34117697e-01],
        [2.61265307e-01, 1.13508052e+00],
        [4.11241843e-01, 6.61542598e-01],
        [4.90586839e-01, 5.73625476e-01],
        [5.68712882e-01, 4.57877612e-01],
        [1.09471625e+00, 6.22598303e-01],
        [1.13245657e+00, 6.84746224e-01],
        [1.45614001e+00, 5.88209935e-01],
        [1.73193957e+00, 4.64255975e-01],
        [1.75275729e+00, 5.19892684e-01],
        [2.19503041e+00, 4.78369936e-01],
        [2.27490372e+00, 7.77336075e-01],
        [2.40270960e+00, 1.79777314e+00],
        [8.79966304e-04, 9.77196660e-01],
        [3.53399491e-02, 2.55506029e+00],
        [1.85316485e-01, 1.48912892e+00],
        [2.64661482e-01, 1.29122794e+00],
        [3.42787524e-01, 1.03068011e+00],
        [8.68790889e-01, 1.40146552e+00],
        [9.06531211e-01, 1.54136016e+00],
        [1.23021465e+00, 1.32405748e+00],
        [1.50601422e+00, 1.04503777e

now we need to do inter-file structure match, first mash all the above into a single function (temporarily)

## Image Debug Area

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def cvshow(name: str, image: np.ndarray):
    cv.namedWindow(name, cv.WINDOW_NORMAL)
    cv.resizeWindow(name, 1000, 1000)
    cv.moveWindow(name, 130, 20)
    cv.imshow(name, image)

In [ ]:
timg = cv.cvtColor(
    (refe_gray * (np.iinfo(np.uint8).max / np.iinfo(np.uint16).max)).astype(np.uint8),
    cv.COLOR_GRAY2BGR,
)
cv.drawContours(timg, refe_contours, -1, (0, 0, 255), cv.FILLED)

In [ ]:
timg = cv.cvtColor(
    (refe_gray * (np.iinfo(np.uint8).max / np.iinfo(np.uint16).max)).astype(np.uint8),
    cv.COLOR_GRAY2BGR,
)
for c in tuple(e[0] for e in refe_stars):
    cv.drawMarker(timg, tuple(int(e) for e in c), (0, 0, 255))

In [ ]:
# cvshow('refe_photo', refe_photo)
# cvshow('refe_photo_g', refe_photo_g)
# [1844:2044, 1844:2044]
# [1744:2144, 1744:2144]
# [1444:2444, 1444:2444]


cvshow('gray', refe_gray[2500:2550, 125:175])
cvshow('wlred', refe_wlred[2500:2550, 125:175])
cvshow('timg', timg[2500:2550, 125:175])


cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
cv.waitKey(0)
cv.destroyAllWindows()